In [2]:
import os
import radon

In [3]:
symmetrics_folter = 'symmetric'
python_suffix = '.py'

file_names = os.listdir(symmetrics_folter)
files = {}
for file_name in file_names:
    with open(os.path.join(symmetrics_folter, file_name), 'r') as file:
        name = file_name.removesuffix(python_suffix)
        files[name] = file.read()
        
list(files.keys())

['tea', 'rcaesar', 'otp', 'vignere', 'caesar', 'feistel']

In [4]:
from radon.complexity import cc_rank, cc_visit
from radon.metrics import h_visit

def get_function_statistics(code: str, function_name: str) -> dict | None:
    # Run analyzers
    cc = cc_visit(code) # Cyclomatic complexity
    h = h_visit(code) # Halstead metrics
    
    # Find CC function
    cc_function = list(filter(lambda x: x.name == function_name, cc))
    if len(cc_function) != 1:
        return None
    cc_function = cc_function[0]
    
    # Find Halstead function
    h_function = list(filter(lambda x: x[0] == function_name, h.functions))
    if len(h_function) != 1:
        return None
    h_function = h_function[0][1] # Get the function's metrics object
    
    # Collect statistics
    stats = {
        'lines': cc_function.endline - cc_function.lineno,
        'cc': cc_function.complexity,
        'cc_rank': cc_rank(cc_function.complexity),
        'h_volume': h_function.volume,
        'h_length': h_function.length,
        'h_difficulty': h_function.difficulty,
    }
    return stats

In [5]:
statistics = {}
for name, code in files.items():
    statistics[ name ] = get_function_statistics(code, 'encrypt')

statistics[ list(statistics.keys())[0] ]

{'lines': 11,
 'cc': 2,
 'cc_rank': 'A',
 'h_volume': 297.25177862321254,
 'h_length': 60,
 'h_difficulty': 3.8461538461538463}

In [6]:
# Create markdown table
table = '| Name | Lines | CC | CC Rank | Halstead Volume | Halstead Length | Halstead Difficulty |\n'
table += '|-|-|-|-|-|-|-|\n'
for name, stats in statistics.items():
    table += f'| {name} | {stats["lines"]} | {stats["cc"]} | {stats["cc_rank"]} | {stats["h_volume"]:.1f} | {stats["h_length"]} | {stats["h_difficulty"]:.1f} |\n'
    
print(table)

| Name | Lines | CC | CC Rank | Halstead Volume | Halstead Length | Halstead Difficulty |
|-|-|-|-|-|-|-|
| tea | 11 | 2 | A | 297.3 | 60 | 3.8 |
| rcaesar | 13 | 3 | A | 98.1 | 24 | 2.5 |
| otp | 4 | 3 | A | 15.5 | 6 | 1.0 |
| vignere | 15 | 3 | A | 80.0 | 21 | 1.9 |
| caesar | 11 | 3 | A | 51.9 | 15 | 1.9 |
| feistel | 8 | 3 | A | 66.6 | 18 | 1.8 |

